# Final Project for Data Acquisition Course
# Football Match, Acquisition-Extraction-Labeling
## by\ Salma Wael , Hania Khaled

## Table of Contents
<ul>
    <li>Video Capture and framing</li>
    <li>Region Labeling</li>
    <li>Feature Extraction and the Blue Team Labeling</li>
    <li>Feature Extraction and the Yellow Team Labeling</li>
    <li>Ball Labeling</li>
    <li>Statistical Data Dense Counting</li>
    
</ul>

<hr>

## Used libraries

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import math

<hr>

## Video Capture and framing

In [ ]:
vido = cv2.VideoCapture('DataExample.mp4')

In [ ]:
video_frames_count = int(vido.get(cv2.CAP_PROP_FRAME_COUNT))
print('Video frames count = ', video_frames_count)

In [ ]:
for i in range(video_frames_count-5000):  # get just first 187 frames
  ret, frame = vido.read()
  if ret:
    frames_folder = f"frames/frame_{i}.jpg"
    cv2.imwrite(frames_folder, frame)

# Close video file
vido.release()

<hr>

## Convert image to hsv colour mode and define colours range

In [ ]:
frame = f"frames/frame_150.jpg"

image = cv2.imread(frame)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

imgs_array = [image, hsv]
plt.figure(figsize=(15,15))
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(imgs_array[i])
plt.show()

In [ ]:
#blue range for detect blue team
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])
#Yellow range for detect yellow team
lower_yellow = np.array([70,70,150])
upper_yellow = np.array([110,250,500])
#white range for detect white ball
lower_white = np.array([145, 163, 113])
upper_white = np.array([255, 255, 238])
#Line white range for detect white lines
lower_white_line = (116,133,89)
upper_white_line = (145,164,119)

<hr>

## Region Labeling

In [ ]:
for i in range(video_frames_count-5000):
    img = cv2.imread(f'frames/frame_{i}.jpg')
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    white_line_mask = cv2.inRange(img2, lower_white_line, upper_white_line)
    plt.imshow(white_line_mask)
    plt.show()

<hr>

## Feature Extraction and the Blue Team Labeling

In [ ]:
blueTeamImage = image
blue = cv2.inRange(hsv, lower_blue, upper_blue)
plt.subplot(1, 2, 1)
plt.imshow(blueTeamImage)
plt.subplot(1, 2, 2)
plt.imshow(blue)
plt.show()

In [ ]:
(contours, hierarchy) = cv2.findContours(blue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
centers = [None]*len(contours)
radius = [None]*len(contours)
for i, c in enumerate(contours):
  contours_poly[i] = cv2.approxPolyDP(c, 3, True)
  centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])
  radius[i]=20

In [ ]:
for i in range(len(contours)):
    cv2.circle(blueTeamImage, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), (50,50,255), 2)
    cv2.circle(blue, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), (50,50,255), 2)
img_array=[blueTeamImage,blue]
plt.figure(figsize=(15,15))
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(img_array[i])
    plt.title("blue team")
plt.show()

<hr>

## Feature Extraction and the Yellow Team Labeling

In [ ]:
yelloTeamImage = image
yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
plt.subplot(1, 2, 1)
plt.imshow(yelloTeamImage)
plt.subplot(1, 2, 2)
plt.imshow(yellow)
plt.show()

In [ ]:
(contours, hierarchy) = cv2.findContours(yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
centers = [None]*len(contours)
radius = [None]*len(contours)
for i, c in enumerate(contours):
  contours_poly[i] = cv2.approxPolyDP(c, 3, True)
  centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])
  radius[i]=20

In [ ]:
for i in range(len(contours)):
    cv2.circle(image, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), (255,255,0), 2)
    cv2.circle(yellow, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), (255,255,0), 2)
img_array=[image,yellow]
plt.figure(figsize=(15,15))
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(img_array[i])
    plt.title("yellow team")
plt.show()

<hr>

## Ball Labeling

In [ ]:
frames = []
for i in range(76,video_frames_count-5000):
  img_file = f'frames/frame_{i}.jpg'
  img = cv2.imread(img_file)
  if img is None:
        continue
  img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  white_mask = cv2.inRange(img2, lower_white, upper_white)
  frames.append(white_mask)

xball= 262
yball= 143
threshold= 15

ball_frames= []
for frame in frames:
    ball_mask= np.copy(frame)
    for r in range(ball_mask.shape[0]):
        for c in range(ball_mask.shape[1]):
            if ball_mask[r][c] > 0:
                distance= math.sqrt((yball-r)**2 +(xball-c)**2)
                if distance <= threshold:
                    yball= r
                    xball= c
                else:
                    ball_mask[r][c] = 0
    ball_frames.append(ball_mask)

for i in range(len(frames)):
    plt.imshow(ball_frames[i])
    plt.show()

<hr>

## Statistical Data Dense Counting

In [ ]:
img = cv2.imread("frames/frame_0.jpg")
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
crops = []
width = img2.shape[1]
height = img2.shape[0]
cx = width // 4
cy = height // 3

# Set the blue and yellow players positions
blue_team = [(2, 132), (45, 81), (152, 27), (193, 155), (299, 104), (324, 120), (290, 77), (337, 37)]
yellow_team = [(406, 55), (399, 99), (435, 125), (482, 201), (240, 230), (303, 149), (313, 113), (284, 94),(210, 55), (160, 133)]

for j in range(3):
    for i in range(4):
        x = i * cx
        y = j * cy

        # Count blue players in crop
        blue_count = 0
        for player in blue_team:
            if player[0] >= x and player[1] >= y and player[0] <= x + cx and player[1] <= y + cy:
                blue_count += 1

        # Count yellow players in crop
        yellow_count = 0
        for player in yellow_team:
            if player[0] >= x and player[1] >= y and player[0] <= x + cx and player[1] <= y + cy:
                yellow_count += 1

        players_count = yellow_count + blue_count
        crops.append({'crop': img2[y:y+cy, x:x+cx],'players_count':players_count, 'blue_count': blue_count, 'yellow_count': yellow_count})

plt.figure(figsize=(12, 9))

for i in range(len(crops)):
    if i < 12:
        plt.subplot(4, 3, i+1)
        plt.imshow(crops[i]['crop'])
        plt.title(f"players: {crops[i]['players_count']}, Blue: {crops[i]['blue_count']}, Yellow: {crops[i]['yellow_count']}")
        plt.axis('off')

plt.show()